In [1]:
from tqdm import tqdm
from src.nlp_dependency_analysis.load_aozora import load_aozora

In [2]:
texts = load_aozora()

In [3]:
texts[:3]

['メロスは激怒した', '必ず、かの邪智暴虐の王を除かなければならぬと決意した', 'メロスには政治がわからぬ']

In [4]:
len(texts)

460

In [9]:
import CaboCha

# https://yu-nix.com/blog/2020/11/11/python-cabocha/
def gen_chunks(tree):
    """
    構文木treeからチャンクの辞書を生成する
    """
    chunks = {}
    key = 0  # intにしているがこれはChunk.linkの値で辿れるようにしている

    for i in range(tree.size()):  # ツリーのサイズだけ回す
        tok = tree.token(i)  # トークンを得る
        if tok.chunk:  # トークンがチャンクを持っていたら
            chunks[key] = tok.chunk  # チャンクを辞書に追加する
            key += 1

    return chunks


def get_surface(tree, chunk):
    """
    chunkからtree内のトークンを得て、そのトークンが持つ表層形を取得する
    """
    surface = ''
    beg = chunk.token_pos  # このチャンクのツリー内のトークンの位置
    end = chunk.token_pos + chunk.token_size  # トークン列のサイズ

    for i in range(beg, end):
        token = tree.token(i)
        surface += token.surface  # 表層形の取得

    return surface

def analyze_cabocha(text: str):
    ret = {}
    cp = CaboCha.Parser()
    tree = cp.parse(text)
    chunks = gen_chunks(tree)
    for from_chunk in chunks.values():
        if from_chunk.link < 0:
            continue  # リンクのないチャンクは飛ばす

        # このチャンクの表層形を取得
        from_surface = get_surface(tree, from_chunk)

        # from_chunkがリンクしているチャンクを取得
        to_chunk = chunks[from_chunk.link]
        to_surface = get_surface(tree, to_chunk)

        ret[from_surface] = to_surface
    return ret

analyze_cabocha("毎日焼肉定食を食べます")

{'毎日': '食べます', '焼肉定食を': '食べます'}

In [10]:
%%time
result = [ analyze_cabocha(t) for t in tqdm(texts) ]

100%|██████████| 460/460 [00:01<00:00, 433.31it/s]

CPU times: user 484 ms, sys: 431 ms, total: 915 ms
Wall time: 1.07 s
